<a href="https://colab.research.google.com/github/raphaelp-silva/deep_learning_com_pytorch_e_python/blob/main/Projeto_9_Regress%C3%A3o_carros_usados_valida%C3%A7%C3%A3o_cruzada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto 9: Regressão carros usados - validação cruzada

## Etapa 1: Importação das bibliotecas

In [1]:
!pip install skorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.9/228.9 kB 13.8 MB/s eta 0:00:00


In [2]:
import pandas as pd
import torch.nn as nn
from skorch import NeuralNetRegressor
import torch
from sklearn.model_selection import cross_val_score
import time
torch.__version__

'2.5.1+cu124'

## Etapa 2: Base de dados

In [3]:
torch.manual_seed(123)

In [4]:
base = pd.read_csv('autos.csv', encoding = 'ISO-8859-1')

In [5]:
base = base.drop('dateCrawled', axis = 1)
base = base.drop('dateCreated', axis = 1)
base = base.drop('nrOfPictures', axis = 1)
base = base.drop('postalCode', axis = 1)
base = base.drop('lastSeen', axis = 1)
base = base.drop('name', axis = 1)
base = base.drop('seller', axis = 1)
base = base.drop('offerType', axis = 1)

In [6]:
base = base[base.price > 10]
base = base.loc[base.price < 350000]

In [7]:
valores = {'vehicleType': 'limousine', 'gearbox': 'manuell',
           'model': 'golf', 'fuelType': 'benzin',
           'notRepairedDamage': 'nein'}
base = base.fillna(value = valores)

In [8]:
previsores = base.iloc[:, 1:13].values
preco_real = base.iloc[:, 0].values.reshape(-1, 1)

In [9]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
onehotencoder = ColumnTransformer(transformers = [("OneHot", OneHotEncoder(), [0,1,3,5,8,9,10])], remainder = 'passthrough')
previsores = onehotencoder.fit_transform(previsores).toarray()

In [10]:
previsores = previsores.astype('float32')
preco_real = preco_real.astype('float32')

## Etapa 3: Construção do modelo

In [17]:
class regressor_torch(nn.Module):
    def __init__(self):
        super().__init__()
        self.dense0 = nn.Linear(314, 158)
        self.dense1 = nn.Linear(158, 158)
        self.dense2 = nn.Linear(158, 1)
        self.activation = nn.ReLU()

    def forward(self, X):
        X = self.dense0(X)
        X = self.activation(X)
        X = self.dense1(X)
        X = self.activation(X)
        X = self.dense2(X)
        return X

In [18]:
regressor_sklearn = NeuralNetRegressor(module = regressor_torch,
                                       criterion = torch.nn.L1Loss,
                                       optimizer = torch.optim.Adam,
                                       max_epochs = 100,
                                       batch_size = 300,
                                       train_split = False)

## Etapa 4: Validação cruzada

In [19]:
resultados = cross_val_score(regressor_sklearn, previsores, preco_real, cv = 5,
                             scoring = 'neg_mean_absolute_error')

  epoch    train_loss     dur
-------  ------------  ------
      1     4103.7635  2.3319
      2     3589.8935  2.9639
      3     3188.2815  2.6493
      4     3025.0123  2.4414
      5     2960.8876  2.4522
      6     2950.7207  2.4910
      7     2893.0678  3.2657
      8     2875.6276  2.4734
      9     2871.2211  2.4702
     10     2822.9350  2.5382
     11     2802.8445  2.8230
     12     2776.6138  2.8913
     13     2755.1381  2.4769
     14     2750.7410  2.5122
     15     2725.4864  2.4794
     16     3324.0727  3.1941
     17     3182.7799  2.4156
     18     2780.8728  2.5042
     19     2747.4493  2.5204
     20     2709.3517  2.5853
     21     2672.6613  3.2140
     22     2655.6054  2.5584
     23     2639.8038  2.5161
     24     2614.2661  2.5408
     25     2599.6697  3.1945
     26     2585.5229  2.7177
     27     2562.6375  2.5979
     28     2554.7761  2.5197
     29     2590.2302  2.6924
     30     2549.8095  3.2933
     31     2510.7881  2.6045
     32   

In [20]:
media = resultados.mean()
desvio = resultados.std()

In [21]:
media

-2344.43935546875

In [22]:
desvio

44.861341404137086